# Domaci - matrica kamere

### 1. Funkcija ParametriKamere(T) koja za datu 3x4 matricu kamere T odredjuje parametre kamere: K,A,C

In [298]:
import numpy as np
from numpy import linalg as la
np.set_printoptions(suppress = True)

In [302]:
def ParametriKamere(T):
    #C(c1, c2, c3, c4) trazimo iz jednakosti TC = 0
    
    #racunamo determinante minora
    c1 = la.det([[T[0][1], T[0][2], T[0][3]],
                 [T[1][1], T[1][2], T[1][3]],
                 [T[2][1], T[2][2], T[2][3]]])
    
    c2 = -la.det([[T[0][0], T[0][2], T[0][3]],
                  [T[1][0], T[1][2], T[1][3]],
                  [T[2][0], T[2][2], T[2][3]]])
    
    c3 = la.det([[T[0][0], T[0][1], T[0][3]],
                 [T[1][0], T[1][1], T[1][3]],
                 [T[2][0], T[2][1], T[2][3]]])
    
    c4 = -la.det([[T[0][0], T[0][1], T[0][2]],
                  [T[1][0], T[1][1], T[1][2]],
                  [T[2][0], T[2][1], T[2][2]]])

    #normiramo po c4
    c1 = np.round(c1/c4)
    c2 = np.round(c2/c4)
    c3 = np.round(c3/c4)
    c4 = 1.0
    
    T0 = np.array(T)
    T0 = T0[:, :3]
    
    #zelimo da vazi T0>0 i det(T0)>0
    if la.det(T0) < 0:
        T0 = np.array(-T)
        T0 = T0[:, :3]
    
    #radimo QR dekompoziciju matrice. Matrica K ce biti R-1 a A ce biti Q-1
    [Q, R] = la.qr(la.inv(T0))
    
    #proveravamo da li je neki od elemenata na dijaognali matrice R negativan
    #ukoliko jeste, mnozimo sa -1 odgovarajucu kolonu matrice Q i vrstu matrice R
    if R[0, 0] < 0:
        R[0, :] = -R[0, :]
        Q[:, 0] = -Q[:, 0]
        
    if R[1, 1] < 0:
        R[1, :] = -R[1, :]
        Q[:, 1] = -Q[:, 1]
 
    if R[2, 2] < 0:
        R[2, :] = -R[2, :]
        Q[:, 2] = -Q[:, 2]
    
    
    K = la.inv(R)
    K = K/K[2][2]
    A = la.inv(Q)
    
    print("\nMatrica K:\n", K)
    print("\nMatrica A:\n", A)
    C = np.array([c1, c2, c3])
    print("\nC:\n", C)

In [303]:
#Provera za test primer sa sajta
n = 11
ParametriKamere([[5,-1-2*n, 3, 18-3*n],
                 [0,-1,5,21],
                 [0,-1,0,1]
                ])


Matrica K:
 [[ 5.  3. 23.]
 [ 0.  5.  1.]
 [ 0.  0.  1.]]

Matrica A:
 [[ 1.  0.  0.]
 [ 0.  0.  1.]
 [-0. -1. -0.]]

C:
 [10.  1. -4.]


In [304]:
#Provera za moj broj indeksa (21/2017, n = 1)
n = 1
ParametriKamere([[5,-1-2*n, 3, 18-3*n],
                 [0,-1,5,21],
                 [0,-1,0,1]
                ])


Matrica K:
 [[5. 3. 3.]
 [0. 5. 1.]
 [0. 0. 1.]]

Matrica A:
 [[ 1.  0.  0.]
 [ 0.  0.  1.]
 [-0. -1. -0.]]

C:
 [-0.  1. -4.]


In [294]:
#funkcija kojom na osnovu svakog para originala i odgovarajuce slike odredjuemo dve jednacine
def cameraEquations(o, s):
    m = np.matrix([[0, 0, 0,0, -s[2]*o[0], -s[2]*o[1], -s[2]*o[2],  -s[2]*o[3], s[1]*o[0], s[1]*o[1], s[1]*o[2],s[1]*o[3]],
                   [s[2]*o[0], s[2]*o[1], s[2]*o[2], s[2]*o[3], 0, 0, 0,0, -s[0]*o[0], -s[0]*o[1], -s[0]*o[2], -s[0]*o[3]]])
    return m

In [295]:
def CameraDLP(originali, projekcije):
    matrica = []
    #na osnovu svih 6 parova pravimo matricu 12x12
    for i in range(6):
        if i > 0:
            m = cameraEquations(originali[i], projekcije[i])
            matrica = np.concatenate((matrica, m), axis=0)
        else:
            matrica = cameraEquations(originali[i], projekcije[i])
    #radimo SVD dekompoziciju matrice
    U, D, Vt = la.svd(matrica)
    
    #matrica P ce biti poslednja kolona matrice V sto je ustvari poslednja vrsta matrica Vt
    #skaliramo tako da element na poziciji 0,0 bude 1
    P = Vt[-1] / Vt[-1, 0]
    P = P.round(4)
    P = P.reshape(3,4)
   
    return P
   

In [296]:
#Provera za test primer sa sajta
n = 11

M1 = np.array([460, 280, 250, 1])
M2 = np.array([50, 380, 350, 1])
M3 = np.array([470, 500, 100, 1])
M4 = np.array([380, 630, 50 * n, 1])
M5 = np.array([30 * n, 290, 0, 1])
M6 = np.array([580, 0, 130, 1])
originali = np.array([M1, M2, M3, M4, M5, M6])

M1p = np.array([288, 251, 1])
M2p = np.array([79, 510, 1])
M3p = np.array([470, 440, 1])
M4p = np.array([520, 590, 1])
M5p = np.array([365, 388, 1])
M6p = np.array([365, 20, 1])
projekcije = np.array([M1p, M2p, M3p, M4p, M5p, M6p])

P = CameraDLP(originali, projekcije)
print(P)


[[     1.        -10.98       33.956   -9692.0317]
 [    26.7404    -10.7543     34.2971 -20328.2746]
 [     0.0417     -0.0073      0.0722    -46.8369]]


In [297]:
#Provera za moj broj indeksa (21/2017, n = 1)
n = 1

M1 = np.array([460, 280, 250, 1])
M2 = np.array([50, 380, 350, 1])
M3 = np.array([470, 500, 100, 1])
M4 = np.array([380, 630, 50 * n, 1])
M5 = np.array([30 * n, 290, 0, 1])
M6 = np.array([580, 0, 130, 1])
originali = np.array([M1, M2, M3, M4, M5, M6])

M1p = np.array([288, 251, 1])
M2p = np.array([79, 510, 1])
M3p = np.array([470, 440, 1])
M4p = np.array([520, 590, 1])
M5p = np.array([365, 388, 1])
M6p = np.array([365, 20, 1])
projekcije = np.array([M1p, M2p, M3p, M4p, M5p, M6p])

P = CameraDLP(originali, projekcije)
print(P)

[[   1.        0.0745   -5.0679 2018.1865]
 [  -1.9611    3.0481   -0.9949 1373.4418]
 [  -0.0011   -0.0026   -0.0038    6.4399]]
